In [9]:
# from llama_index.llms.openai import OpenAI
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex 

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings 
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
import os
import nest_asyncio

from dotenv import load_dotenv
load_dotenv()
nest_asyncio.apply()

# llm = OpenAI(model="gpt-4o-mini")
llm = GoogleGenAI(model="gemini-2.0-flash")

In [10]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [11]:
# Index global - chỉ khởi tạo 1 lần
index_dsdaihoc = LlamaCloudIndex(
        name="dsdaihoc",
        project_name="Default",
        organization_id="1bcf5fb2-bd7d-4c76-b6d5-bb793486e1b3",
        api_key=os.getenv("LLAMA_CLOUD_API_KEY"),
        )


# 2️⃣ Setup tools
qe = index_dsdaihoc.as_query_engine(similarity_top_k=5)

In [12]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=qe,
        metadata=ToolMetadata(
            name="dsdaihoc",
            description="Danh sách các trường đại học ở việt nam",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [ ]:

response = query_engine.query(
    "Học trường kinh tế nào ở Hà Nội, học phí dưới 2 triệu/tháng, ngành tài chính ngân hàng có triển vọng khi ra trường ",
)

Generated 1 sub questions.
[dsdaihoc] Q: Danh sách các trường đại học ở Hà Nội
**********
Trace: query
    |_CBEventType.QUERY -> 8.580246 seconds
      |_CBEventType.SYNTHESIZE -> 2.250287 seconds
        |_CBEventType.TEMPLATING -> 1.1e-05 seconds
        |_CBEventType.LLM -> 2.231829 seconds
**********
[dsdaihoc] A: Đại học Quốc gia Hà Nội (VNU), Trường Đại học Sư phạm Hà Nội 2 (HPU2), Trường Đại học Mở Hà Nội (HOU), Học Viện Chính Sách và Phát Triển (APD)
**********
Trace: query
    |_CBEventType.QUERY -> 8.580246 seconds
      |_CBEventType.SYNTHESIZE -> 2.250287 seconds
        |_CBEventType.TEMPLATING -> 1.1e-05 seconds
        |_CBEventType.LLM -> 2.231829 seconds
**********


TypeError: object Response can't be used in 'await' expression

In [14]:

print(response)

Trường Đại học Mở Hà Nội (HOU) là một trong những trường có chương trình kinh tế với học phí dưới 2 triệu/tháng ở Hà Nội. Ngành tài chính ngân hàng tại trường này có triển vọng khi ra trường.


In [15]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: Danh sách các trường đại học ở Hà Nội
Answer: Đại học Quốc gia Hà Nội (VNU), Trường Đại học Sư phạm Hà Nội 2 (HPU2), Trường Đại học Mở Hà Nội (HOU), Học Viện Chính Sách và Phát Triển (APD)


In [ ]:
import os
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

# 1. Thiết lập API key
os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv("LLAMA_CLOUD_API_KEY", "")

def ask(question: str) -> str:
    # 2. Kết nối đến chỉ mục LlamaCloud đã tạo
    index = LlamaCloudIndex(
        name="dsdaihoc",
        project_name="Default",
        organization_id="1bcf5fb2-bd7d-4c76-b6d5-bb793486e1b3",
    )
    
    # 3. Tạo QueryEngine: bao gồm retrieval + generation
    query_engine = index.as_query_engine(similarity_top_k=3)
    
    # 4. Gửi câu hỏi, nhận kết quả
    response = query_engine.query(question)
    return response.response  # hoặc .text nếu SDK dùng vậy

if __name__ == "__main__":
    q = input("Bạn hỏi: ").strip()
    ans = ask(q)
    print("\n👉 AI trả lời:", ans)


**********
Trace: query
    |_CBEventType.RETRIEVE -> 1.10677 seconds
**********


AttributeError: 'LlamaCloudIndex' object has no attribute 'query'